

# Relax interface using EMT potentials

Optimize atoms coordinates of the interface using the EMT potential and BFGS optimizer. The relaxation is performed to minimize the energy of the interface.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set relaxation parameters in cell 2.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Perform relaxation of the interface with set parameters
1. View the structure before and after relaxation

## Notes
1. We perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered.
1. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. 
1. ZSL strain matching is performed using Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).
1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [1]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 
# Maximum area for the superlattice search algorithm
MAX_AREA = 50
FMAX = 0.05

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [2]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

### 1.3. Get input interface material
Materials are loaded with `get_data()`.

In [3]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
interface = materials[2]

ModuleNotFoundError: No module named 'mat3ra.esse.models'

### 1.4. Preview Interface

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize(interface, repetitions=[3, 3, 1], rotation="0x")

## 2. Perform Relaxation
### 2.1. Set the optimization parameters

In [ ]:
from ase.optimize import BFGS
from ase.calculators.emt import EMT
from mat3ra.made.tools.convert import to_ase

calculator = EMT()
optimizer = BFGS
OPTIMIZATION_PARAMETERS = {
    "FMAX": FMAX
}

### 2.2. Optimize atomic coordinates of the interface and view energy/step plot

In [ ]:
from utils.plot import create_realtime_plot, plot_update_callback
from mat3ra.made.tools.convert import from_ase

# Add calculator to the interface for relaxation
ase_interface = to_ase(interface)
ase_interface.set_calculator(calculator)

dyn = optimizer(ase_interface)
steps = []
energies = []

fig = create_realtime_plot()

# Run the relaxation
dyn.attach(plot_update_callback(dyn, ase_interface, fig, steps, energies), interval=1)
dyn.run(fmax=OPTIMIZATION_PARAMETERS["FMAX"])

ase_final_interface = ase_interface
final_interface = Material(from_ase(ase_final_interface))

### 5.3. View structure before and after relaxation

In [ ]:
visualize([{"material": interface, "title": "original"}, {"material":final_interface, "title":"relaxed"}], rotation= "-90x", repetitions=[3, 3, 1])


### 5.4. Calculate energy using ASE EMT
The interfacial energy is the sum of the surface energies of the substrate and film minus the adhesion energy. According to Dupré's formula

In [ ]:
from mat3ra.made.tools.calculate import calculate_total_energy, calculate_interfacial_energy

original_energy = calculate_total_energy(interface, calculator)
relaxed_energy = calculate_total_energy(final_interface, calculator)

interfacial_energy = calculate_interfacial_energy(interface=final_interface, substrate_bulk=substrate, film_bulk=film,  calculator=calculator)

# Print out the metrics
print(f"Starting interface energy: {original_energy:.4f} eV")
print(f"Relaxed interface energy: {relaxed_energy:.4f} eV")
print(f"Interfacial energy: {interfacial_energy:.4f} eV")

## 6. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_data

set_data("materials", [final_interface.to_json()])